# softmax回归的简洁实现

我们在[“线性回归的简洁实现”](linear-regression-gluon.ipynb)一节中已经了解了使用Gluon实现模型的便利。下面，让我们再次使用Gluon来实现一个softmax回归模型。首先导入所需的包或模块。

In [1]:
%matplotlib inline
import d2lzh as d2l
from mxnet import gluon, init
from mxnet.gluon import loss as gloss, nn
import tensorflow as tf

## 获取和读取数据

我们仍然使用Fashion-MNIST数据集和上一节中设置的批量大小。

In [2]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

train, test = tf.keras.datasets.fashion_mnist.load_data()
images, labels = train
images = images.astype('float32')
x_test, y_test = test
x_test = x_test.astype('float32')
images = images/255.0
labels = labels.astype('int32')
x_test = x_test/255.0
y_test = y_test.astype('int32')
fmnist_train_ds = tf.data.Dataset.from_tensor_slices((images, labels))
fmnist_train_ds = fmnist_train_ds.shuffle(5000).batch(batch_size)

## 定义和初始化模型

在[“softmax回归”](softmax-regression.ipynb)一节中提到，softmax回归的输出层是一个全连接层。因此，我们添加一个输出个数为10的全连接层。我们使用均值为0、标准差为0.01的正态分布随机初始化模型的权重参数。

In [3]:
net = nn.Sequential()
net.add(nn.Dense(10))
net.initialize(init.Normal(sigma=0.01))

model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Softmax())
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                7850      
_________________________________________________________________
softmax (Softmax)            (None, 10)                0         
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


## softmax和交叉熵损失函数

如果做了上一节的练习，那么你可能意识到了分开定义softmax运算和交叉熵损失函数可能会造成数值不稳定。因此，Gluon提供了一个包括softmax运算和交叉熵损失计算的函数。它的数值稳定性更好。

In [4]:
loss = gloss.SoftmaxCrossEntropyLoss()
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

## 定义优化算法

我们使用学习率为0.1的小批量随机梯度下降作为优化算法。

In [5]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.1})
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

In [6]:
model.compile(optimizer=optimizer,
              loss=loss_fn,
              metrics=['accuracy'])

## 训练模型

接下来，我们使用上一节中定义的训练函数来训练模型。

In [7]:
num_epochs = 5
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, None,
              None, trainer)

epoch 1, loss 0.7909, train acc 0.746, test acc 0.806
epoch 2, loss 0.5719, train acc 0.812, test acc 0.821
epoch 3, loss 0.5296, train acc 0.823, test acc 0.829
epoch 4, loss 0.5046, train acc 0.830, test acc 0.836
epoch 5, loss 0.4893, train acc 0.835, test acc 0.840


In [8]:
model.fit(fmnist_train_ds, epochs=num_epochs)
model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
235/235 [==============================] - 1s 4ms/step - loss: 0.8021 - accuracy: 0.7351
Epoch 2/5
235/235 [==============================] - 0s 2ms/step - loss: 0.5743 - accuracy: 0.8113
Epoch 3/5
235/235 [==============================] - 0s 2ms/step - loss: 0.5279 - accuracy: 0.8248
Epoch 4/5
235/235 [==============================] - 0s 2ms/step - loss: 0.5034 - accuracy: 0.8316
Epoch 5/5
235/235 [==============================] - 0s 2ms/step - loss: 0.4868 - accuracy: 0.8359
10000/1 - 0s - loss: 0.4075 - accuracy: 0.8283


[0.505410792684555, 0.8283]

## 小结

* Gluon提供的函数往往具有更好的数值稳定性。
* 可以使用Gluon更简洁地实现softmax回归。

## 练习

* 尝试调一调超参数，如批量大小、迭代周期和学习率，看看结果会怎样。



## 扫码直达[讨论区](https://discuss.gluon.ai/t/topic/740)

![](../img/qr_softmax-regression-gluon.svg)